# 스파크 스트리밍 실습 7교시 : 참고할 사항

> 성능 튜니에 대한 기본적인 가이드와 스트리밍 애플리케이션에서 사용되는 용어를 정리합니다


## 목차
* [1. 성능 개선 방향](#1.-성능-개선-방향)
* [2. 자주 사용되는 용어](#2.-자주-사용되는-용어)
* [3. 예상되는 질문들](#3.-예상되는-질문들)

## 1. 성능 개선 방향

> 배치 작업과 다르게 '마이크로 배치' 잡들은 비교적 작은 볼륨의 데이터를 다루기 때문에, 살짝 다른 튜닝 접근을 하고 있습니다.

* 클러스터 리소스 프로비저닝 - Cluster resource provisioning
  - 24/7 서비스를 수행하기 위해 적절한 리소스 프로비저닝이 필수적입니다. 다만, 너무 많은 리소스 할당은 낭비를 초래하고, 적으면 작업이 실패하게 됩니다
  - ***'스테이트리스' 쿼리들은 코어***가 많이 필요지만, ***'스테이트풀' 쿼리들은 상대적으로 메모리***를 많이 필요하기 때문에 쿼리의 특성에 따라 할당하는 것을 고려합니다

* 셔플에 따른 파티션 수 - Number of partitions for shuffles
  - 배치 작업 대비 다소 작은 셔플 파티션의 수를 가지는데, 너무 많은 작업으로 구분하는 것에 따른 오버헤드를 증가시키거나, 처리량을 감소시킬 수 있습니다
  - 또한 셔플링은 '스테이트풀 오퍼레이션'에 따른 '체크포인팅'의 오버헤드가 커질 수 있다는 점도 유의할 필요가 있습니다
  - '스테이트풀' 작업이면서 트리거 간격이 몇 초에서 몇 분인 스트리밍 쿼리의 경우 ***셔플 수를 기본값 인 200에서 할당 된 코어 수의 최대 2~3배로 조정***하는 것이 좋습니다.

* 안정화를 위한 소스 비율 리미트 조정 - Setting source rate limits for stability
  - 초기에 최적화된 설정으로 잘 운영되더라도, 급격하게 늘어난 데이터에 대한 불안정한 상황이 발생할 수 있는데, 리소스를 많이 투입하는 '오버 프로비저닝' 방법 외에도 '소스 속도 제한'을 통한 불안정성을 극복할 수도 있습니다. 
  - 카프카 등의 소스의 임계치를 설정함으로써 싱글 '마이크로 배치' 작업에서 너무 많은 데이터를 처리하지 않도록 막는 방법이며, 증가한 데이터는 소스에서 버퍼링 된 상태로 유지되며, 결국에는 데이터 처리가 따라잡게 됩니다
  - 다만, 아래의 몇 가지를 기억해야만 합니다
    - 리미트를 너무 낮게 설정하면 쿼리가 할당 된 리소스를 충분히 활용하지 못하고 입력 속도보다 느려질 수 있습니다.
    - 리미트 설정 만으로는 입력 속도의 지속적인 증가를 효과적으로 해결하지 못하며, 안정성이 유지되는 동안 버퍼링되고 처리되지 않은 데이터의 양은 소스에서 무한히 증가하므로 종단 간 지연 시간도 증가한다는 점을 알아야 합니다

* 동일한 스파크 애플리케이션 내에서 다수의 스트리밍 쿼리 - Multiple streaming queries in the same Spark application
  - 동일한 SparkContext 또는 SparkSession에서 여러 스트리밍 쿼리를 실행하면 fine-grained 된 리소스 공유가 발생할 수 있습니다.
    - 각 쿼리를 실행하면 Spark 드라이버 (즉, 실행중인 JVM)의 리소스가 계속 사용됩니다. 결국, 드라이버가 동시에 실행할 수있는 쿼리 수가 제한되게 됩니다.
      - 이러한 제한에 도달하면 작업 예약에 병목 현상이 발생하거나 (즉, 실행 프로그램을 제대로 활용하지 못함) 메모리 제한을 초과 할 수 있습니다.
    - 별도의 스케줄러 풀에서 실행되도록 설정하여 동일한 컨텍스트의 쿼리간에보다 공정한 리소스 할당을 보장 할 수 있습니다.
      - SparkContext의 스레드 로컬 속성 spark.scheduler.pool을 각 스트림에 대해 다른 문자열 값으로 설정합니다.

```python
# Run streaming query1 in scheduler pool1
spark.sparkContext.setLocalProperty("spark.scheduler.pool", "pool1")
df.writeStream.queryName("query1").format("parquet").start(path1)

# Run streaming query2 in scheduler pool2
spark.sparkContext.setLocalProperty("spark.scheduler.pool", "pool2")
df.writeStream.queryName("query2").format("parquet").start(path2)
```

## 2. 자주 사용되는 용어
* Structured API 란?
  - 일반적인 함수 혹은 메소드 형식의 API 와 다르게, 데이터프레임을 통하여 다양한 메소드를 연결하여 활용할 수 있는 구조를 가졌기 때문에 구조화된 API 라고 말할 수 있습니다
* Map-Reduce 란?
  - 병렬처리가 가능한 상태가 없는 단순 계산 연산을 Map, 병렬처리가 불가능한 상태가 존재하는 집계 연산을 Reduce 라고말할 수 있으며, 가장 쉬운 예제로 WordCount 가 있습니다
* Idempotence 란?
  - 반복적으로 수행 혹은 처리하여도 항상 결과가 동일하게 나오는 것
* Structured Streaming vs. DStreams ?
  - DStream : RDD 를 이용하여 스트리밍 처리를 직접 구현할 수 있는 Spark 1.x 버전부터 제공 되었던 인터페이스를 말합니다
  - Structured Streaming : Spark 2.x 버전부터 제공 되는 Spark SQL 즉, Dataframe (Dataset) 기반의 API 통한 스트리밍 처리 인터페이스를 말합니다
* [RDD vs. Dataset vs. Dataframe](https://www.analyticsvidhya.com/blog/2020/11/what-is-the-difference-between-rdds-dataframes-and-datasets/)
  - RDD : 최적화 및 데이터 처리에 대한 구현을 직접해야 하는 경우
```python
sc = SparkContext("local","PySpark Word Count Exmaple")
words = sc.textFile("tmp/source").flatMap(lambda line: line.split(" "))
wordCounts = words.map(lambda word: (word, 1)).reduceByKey(lambda a,b:a +b)
wordCounts.saveAsTextFile("tmp/target")
```
  - Dataframe : 제공되는 API 통한 데이터 처리를 수행하는 경우
```python
df = spark.read.text("tmp/source")
wordCounts = df.withColumn("word", explode(split("value", " "))).groupBy("word").count()
wordCounts.write.format("text").save("tmp/target")
```
  - Dataset : Typed Compile 언어인 Java, Scala 만 지원하며, Compile 시점에 Type-safe 보장
* Event-time vs. Processing-time ?
  - Event-time : 실제 사건 혹은 이벤트가 발생한 시간 (ex_ 핸드폰에서 광고를 클릭한 시간)
  - Processing-time : 데이터를 수집 혹은 수신한 시간 (ex_ 브로커를 통해 로그가 수신된 시간)
* Bounded vs. Unbounded ?
  - Bounded : 데이터를 처리하는 시점에 그 범위가 명확하게 정해진 경우를 말합니다 (ex_ 배치 처리와 같이 어제 하루에 수신된 모든 로그를 처리해야 하는 경우)
  - Unbounded : 데이터를 처리하는 시점에 그 범위가 명확하게 정해지지 않은 경우를 말합니다 (ex_ 스트리밍 데이터와 같이 끝없이 이어지는 데이터를 수신하고 처리하는 경우)
* Source vs. Sink ?
  - Source : 처리해야 하는 데이터 소스 위치
  - Sink : 데이터를 저장 혹은 전달하는 타겟 위치
* incrementalization ?
  - 스트리밍 처리에서 스파크는 Unbounded 데이터를 마치 정적인 테이블인 것처럼 동작하게 하고, 최종 출력 싱크에 저장될 결과 테이블을 계산합니다.
  - **여기서의 배치와 유사한 쿼리 수행을 스트리밍 실행 계획으로 변환**하게 하는데 이 과정을 '증분화(*incrementalization*)'라고 합니다. 
  - 스파크는 레코드가 도착할 때마다 결과를 업데이트 하기 위해 어떤 상태를 유지해야 하는지를 확인하여 점진적으로 결과를 업데이트 합니다.
  - 즉, 마치 배치 처리를 통한 소스 테이블 쿼리 후 타겟 테이블에 저장하는 것 처럼 보이지만, 실제로는 증분화된 스트리밍 처리 계획이며 이를 증분화라고 합니다
* materializing ?
  - RDD 혹은 Dataframe 의 데이터 객체 상태의 meterialized 의 의미는 lazy evaluation 관점에서 보았을 때에 not materialized 의 의미는 실행 계획이 아직 검토되지 않았다고 말할 수 있으며, materialized 되었다는 의미는 실행계획을 통해 대상 데이터가 메모리에 올라와서 접근가능한 상태정도라고 말할 수 있습니다.
    - we allocate memory for it. That is, we store it into memory (.persist()) or even store it into durable storage (.persist(RELIABLE)).
    - Sparks runtime does lazy evaluation so until an action is taken the RDD is not materialized.
  - 한편으로 논리적인 테이블 관점에서 보았을 때에는, createOrReplaceTempView 는 가상의 테이블에 대한 메타정보만 가진 테이블을 말하며, saveAsTable 명령은 실제 데이터 프레임을 물리적인 저장 경로에 저장한 상태를 말합니다.
    - Unlike the createOrReplaceTempView command, saveAsTable will materialize the contents of the DataFrame and create a pointer to the data in the Hive metastore. 
  - 참고로 스파크에서는 meterialized view 를 아직 지원하지(SPARK-29038) 않습니다
* epoch ?
  - 컴퓨팅 컨텍스트에서 에포크는 컴퓨터의 시계 및 타임스탬프 값이 결정되는 날짜와 시간입니다.
  - 에포크는 일반적으로 시스템마다 다른 특정 날짜의 0시 0분 0초(00:00:00) UTC(협정 세계시)에 해당합니다.
  - 예를 들어 대부분의 Unix 버전은 1970년 1월 1일을 epoch 날짜로 사용합니다. - [Epoch Time](https://searchdatacenter.techtarget.com/definition/epoch)
    - 스트리밍 처리에 있어서 데이터 처리 트랜잭션 관리 및 유지를 위해 commit log 의 commits 정보를 metadata checkpoint directory 에 저장하게 됩니다.
    - `Offset Commit Log — commits Metadata Checkpoint Directory` 체크포인트 경로에 메타데이터를 커밋하는 행위를 말합니다.
* micro-batch ?
  - 스파크의 Structured Streaming 은 엄밀히 말하면 Continuous Streaming 처리가 아니라 500ms 수준의 작은 배치 작업으로 쪼개어 (deterministic) 수행하는데 이 작은 배치 작업을 말합니다
    - Minimum batch size Spark Streaming can use.is 500 milliseconds, is has proven to be a good minimum size for many applications.
    - The best approach is to start with a larger batch size (around 10 seconds) and work your way down to a smaller batch size.
* *stateless* and *stateful* in spark transformation?
  - stateless : 스트리밍 처리에 있어 이전 상태에 의존하지 않고 현재의 상태만 이용하여 변환 혹은 처리를 수행하는 경우 (ex_ 현재 그룹 내의 빈도 혹은 특정 값의 상태)
  - stateful : 스트리밍 처리에 있어 이전 상태에 의존한 데이터 변환을 말합니다 (ex_ 누적 값 혹은 집계 연산 결과 상태)


## 3. 예상되는 질문들

* 스트리밍의 경우 explain 명령을 통해 '로지컬 플랜' 혹은 '최적화된 플랜'을 확인할 수 없나요?
  - 스트리밍 쿼리에 대해 아래와 같이 explain 함수를 통해 확인할 수 있습니다
```python
query.processAllAvailable()
query.explain()
```

* 동일한 키가 존재하는 정렬의 경우 왜 결정론적이지 못 한가요?
  - 분산환경에서 셔플링이 발생하는 경우 일시적인 네트워크 지연, 특정 노드의 장애에 따라 Reduce 노드에 도착하는 데이터의 순서는 언제든지 변경될 수 있습니다

* 스트리밍 처리가 1초 단위라고 했는데 소캣을 통한 테스트 시에는 화면 출력이 아주 느려 보이는데 왜 그런가요?
  - 내부 트리거는 계속 발생하지만, 데이터 소스로부터 가져올 데이터가 없기 때문에 수행되지 않는 것처럼 보입니다
  - 화면에 출력되는 것은 최종 출력 싱크에 트리거로부터 저장할 데이터가 발생하는 경우에만 발생하는 이벤트라서 그렇습니다

* 계속 지켜보면 데이터가 없지만 결과 테이블이 출력되는 경우도 있는데 왜 그런가요?
  - 메시지를 잘 살펴보시면 아래와 같은데요, 해석해 보면, '현재 배치는 지연되고 있으며, 트리거는 1초 이지만 약 8초 정도 소요되었다'라고 나오고 있습니다.
  - `21/06/27 08:19:06 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 1000 milliseconds, but spent 8314 milliseconds`
  - '스파크 스트리밍' 처리에 대한 기준은 몇 가지가 있는데 1: 초기 기동 시에 실행, 2: 소스에 처리할 데이터가 존재하는 경우 실행, 3: 임계시간이 지난경우 

* JSON 혹은 header 가 있는 TSV 등은 스키마 infer 가 가능한데 왜 schema 를 명시적으로 생성해 주어야 하나요?
  - '스파크 스트리밍'의 경우 데이터 타입에 민감하며 자칫 실수하는 경우 전체 스트리밍 파이프라인의 장애로 이어지기 때문에 엄격한 스키마 정의가 반드시 필요합니다
  - spark.readStream.schema(schema) 와 같이 정의되어야만 하며, 그렇지 않으면 아래와 같은 오류를 출력합니다
  - `IllegalArgumentException: Schema must be specified when creating a streaming source DataFrame. If some files already exist in the directory, then depending on the file format you may be able to create a static DataFrame on that directory with 'spark.read.load(directory)' and infer schema from it.`

* 집계함수 출력이 안 되는데 왜 그런가?
  - 기본 출력 모드가 append 모드이기 떄문에, watermark 지정을 해주지 않는 경우 집계함수를 append 모드 통해서 저장할 수 없습니다. (consistency 문제)
  - 출력모드를 console complete 로 지정하여 테스트하거나, watermark 를 통해 출력하여야만 하며 그렇지 않으면 아래와 같은 오류를 출력합니다
  - `AnalysisException: Append output mode not supported when there are streaming aggregations on streaming DataFrames/DataSets without watermark`

* 입력 데이터 지정시에 JSON 파일을 넣을 수는 없는가?
  - '스파크 스트리밍' 데이터는 파일 단위로 생성된 데이터 여부를 판단하기 때문에 경로를 입력해야만 합니다 그렇지 않으면 아래와 같은 오류를 콘솔에 출력합니다
  - `java.lang.IllegalArgumentException: Option 'basePath' must be a directory`
    
* 카프카 엔진의 포트가 왜 9092, 9093 으로 나누어져 있고, kafka:9093, localhost:9092 로 접근해야만 하는가?
  - 카프카 디폴트 포트는 9092 포트입니다
  - 리스너 설정 시에 기본 구성의 경우 `listeners=PLAINTEXT://broker1:9091,SSL://broker1:9092,SASL_SSL://broker1:9093` 으로 9091 ~ 9093 까지 용도에 따라 설정합니다
  - [Security Protocol](http://kafka.apache.org/11/javadoc/org/apache/kafka/common/security/auth/SecurityProtocol.html) - 프로토콜은 아래와 같습니다.
  | Enum Constant | Description |
  | --- | --- |
  | PLAINTEXT | Un-authenticated, non-encrypted channel |
  | SASL_PLAINTEXT | SASL authenticated, non-encrypted channel |
  | SASL_SSL | SASL authenticated, SSL channel |
  | SSL | SSL channel |

* 스파크 스트리밍도 배치 처리와 유사하게 몇 개의 익스큐터가 동시에 기동되고 항상 떠서 서로간의 데이터를 주고 받는 방식인가요?
  - 익스큐터들 내에 집계 연산을 위해 셔플을 통해 데이터를 전송합니다

* 텀블링 윈도우는 항상 그 시간대를 유지하면서 메모리에 저장되어 있는가?
  - 워터마크가 없는 스트리밍은 끝없이 출력되는 것으로 간주하고 수행됩니다


### References
* [Streaming Programming Guide](https://spark.apache.org/docs/latest/streaming-programming-guide.html)
* [Spark Runtime Configuration Guide](https://spark.apache.org/docs/latest/configuration.html#spark-sql)
* 5-2. [Structured Streaming Programming Guide](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#creating-streaming-dataframes-and-streaming-datasets)
* 5-2. [Apache Kafka Client](https://docs.confluent.io/clients-confluent-kafka-python/current/overview.html)
  - [Kafka python on github](https://github.com/dpkp/kafka-python/blob/master/example.py)
  - [Kafka Producer & Consumer](https://needjarvis.tistory.com/607)
* 5-2. [Cassandar Data Manipulation](https://cassandra.apache.org/doc/latest/cql/dml.html)
  - [Creating Cassandra keyspace & table on docker start up](https://www.linkedin.com/pulse/creating-cassandra-keyspace-table-docker-start-up-amon-peter/)
* 5-2. [MySQL JDBC Connection Example](https://dev.mysql.com/doc/connector-python/en/connector-python-examples.html)
* [Linux Kill](https://www.lesstif.com/system-admin/unix-linux-kill-12943674.html)
* [Python Current Timestamp](https://timestamp.online/article/how-to-get-current-timestamp-in-python)
* [NetCat on Windows](https://mentha2.tistory.com/65)
* [NetCat on Ubuntu](https://epicarts.tistory.com/43)
* [VisualVM](https://visualvm.github.io/download.html)
* [Monitoring spark w/ prometheus](https://argus-sec.com/monitoring-spark-prometheus/)
* [Dynamic Docker Monitoriong](https://www.datadoghq.com/dg/monitor/docker-benefits/)